In [1]:
%load_ext autoreload

In [3]:
%autoreload 2
import os
import glob
import guitarpro
import librosa, soundfile
import numpy as np
import scipy
from utils import *
from operations import *

## Break multi-track gtp files into single-track gtp files

In [ ]:
MULTI_TRACK_DIR = "/Volumes/MacOnly/UG_raw/all_time_top_by_hits"
SINGLE_TRACK_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/single_track_gtps"

i = 0
for file in glob.glob(os.path.join(MULTI_TRACK_DIR, "*.gp*")):
    i += 1
    print(f"processing file {i}: {file.split('/')[-1]}")
    get_single_tracks(
        file=file,
        output_dir=SINGLE_TRACK_DIR,
        unify_volume=True,
        force_clean=False,
        disable_repeats=False,
        disable_mixTableChange=False,
    )



In [ ]:
MULTI_TRACK_DIR = "/Volumes/MacOnly/UG_raw/all_time_top_by_hits"
# clean_single_track_gtps contains gtp files that have:
# unified volume, clean guitar tone, no repeats, no mixTableChanges
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)

i = 0
for file in glob.glob(os.path.join(MULTI_TRACK_DIR, "*.gp*")):
    i += 1
    print(f"processing file {i}: {file.split('/')[-1]}")
    get_single_tracks(file=file, output_dir=CLEAN_SINGLE_TRACK_DIR)



## Segment single-track audio into poly and mono segments (GT)

In [4]:
# a test
file_name = "DragonForce - Through The Fire And Flames_Lead 2.gp5"
song = guitarpro.parse(os.path.join(CLEAN_SINGLE_TRACK_DIR, file_name))
poly, mono = poly_vs_mono(song)


In [5]:
poly


[[37.2, 38.400000000000006],
 [288.6, 289.2],
 [289.8, 290.4],
 [414.0, 418.79999999999995]]

In [6]:
mono


[[0.0, 37.2],
 [38.4, 288.59999999999997],
 [289.2, 289.8],
 [290.4, 414.0],
 [418.8, 433.2]]

In [ ]:
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)
SINGLE_TRACK_AUDIO_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_audio"
)
POLY_SEGMENTS_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/poly_audio_segments"
MONO_SEGMENTS_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/mono_audio_segments"

i = 0
for file in glob.glob(os.path.join(CLEAN_SINGLE_TRACK_DIR, "*.gp5")):
    i += 1
    track_title, _ = os.path.splitext(file.split("/")[-1])
    print(f"processing file {i}: {track_title}")

    song = guitarpro.parse(file)
    poly, mono = poly_vs_mono(song)

    y, sr = librosa.load(
        os.path.join(SINGLE_TRACK_AUDIO_DIR, track_title + ".wav"), sr=None, mono=True
    )

    for j, poly_segment in enumerate(poly):
        start_sec = poly_segment[0]
        end_sec = poly_segment[1]
        segment = y[int(start_sec * sr) : int(end_sec * sr)]
        # write segment to file, store it in the "poly" folder
        segment_file_name = "{}_{}_{}s.wav".format(
            track_title, i, int(start_sec)
        )  # this i should actually be j, but it doesn't matter for now
        soundfile.write(os.path.join(POLY_SEGMENTS_DIR, segment_file_name), segment, sr)

    for j, mono_segment in enumerate(mono):
        start_sec = mono_segment[0]
        end_sec = mono_segment[1]
        segment = y[int(start_sec * sr) : int(end_sec * sr)]
        # write segment to file, store it in the "mono" folder
        segment_file_name = "{}_{}_{}s.wav".format(track_title, i, int(start_sec))
        soundfile.write(os.path.join(MONO_SEGMENTS_DIR, segment_file_name), segment, sr)



## Generate frames and labels from the poly and mono segments

In [ ]:
# get a bunch of frames and put them into a csv/npz file
# put corresponding labels in a separate csv file
# shuffle them
# train/test split
# sklearn classifier


In [44]:
# frame-level features: raw waveform, spectrum
from scipy.fft import rfft, rfftfreq

FRAME_SIZE = 2048
HOP_SIZE = 1024
MONO_LABEL = 0

file = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/mono_audio_segments/ACDC - Back In Black (ver 4 by GuitarManiac09)_Angus Young_1_195s.wav"
y, sr = librosa.load(file, sr=None, mono=True)
y = normalize_audio(y)
frames = librosa.util.frame(y, frame_length=FRAME_SIZE, hop_length=HOP_SIZE, axis=0)

features_with_label = []
for frame in frames:
    if np.all(frame == 0):
        continue
    spectrum = np.abs(rfft(frame)) # length: FRAME_SIZE/2 + 1
    feature = np.concatenate((frame, spectrum), axis=0)
    # put the label at the last column, this should make shuffle easier
    feature_with_label = np.append(feature, MONO_LABEL)
    features_with_label.append(feature_with_label)
features_with_label = np.array(features_with_label)
# features_with_label shape: (12, 3074)
# 12 frames,  3074 = 2048(rawSignal) + 1025(halfSpectrum) + 1(label)


In [ ]:
# frame-level features: raw waveform, spectrum
from scipy.fft import rfft, rfftfreq

FRAME_SIZE = 2048
HOP_SIZE = 1024
MONO_LABEL = 0

POLY_SEGMENTS_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/poly_audio_segments"
MONO_SEGMENTS_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/mono_audio_segments"
MONO_SEGMENTS_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/test"

i = 0
features_with_label = []
for file in glob.glob(os.path.join(MONO_SEGMENTS_DIR, "*.wav")):
    i += 1
    title, _ = os.path.splitext(file.split("/")[-1])
    print(f"processing file {i}: {title}")

    y, sr = librosa.load(file, sr=None, mono=True)
    # y = normalize_audio(y)
    frames = librosa.util.frame(y, frame_length=FRAME_SIZE, hop_length=HOP_SIZE, axis=0)
    for frame in frames:
        if np.all(frame == 0):
            print("empty frame kicked out")
            continue
        frame = normalize_audio(frame)
        spectrum = np.abs(rfft(frame)) # length: FRAME_SIZE/2 + 1
        feature = np.concatenate((frame, spectrum), axis=0)
        # put the label at the last column, this should make shuffle easier
        feature_with_label = np.append(feature, MONO_LABEL)
        features_with_label.append(feature_with_label)
features_with_label = np.array(features_with_label)
# features_with_label shape: (n, 3074)
# n frames,  3074 = 2048(rawSignal) + 1025(halfSpectrum) + 1(label)
features_with_label.shape

In [60]:
np.savez_compressed("/Volumes/MacOnly/UG_proc/all_time_top_by_hits/test.npz", features_with_label)
# problem: normalization eliminates zero segments by substracting the mean
# maybe do frame level normalization? normalize only if it's not entirely zeros
# overlapping is probably not necessary?
# larger frame size? 
# run the whole thing on colab, may consume a lot of memory

In [47]:
from sklearn.svm import LinearSVC
model = LinearSVC()
features = features_with_label[:, :-1]
labels = features_with_label[:, -1]

labels[3] = 1
model.fit(X=features, y=labels)

LinearSVC()

## Break the single-track gtp files into 4-bar phrases (necessary??)

In [ ]:
SINGLE_TRACK_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/single_track_gtps"
PHRASE_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/phrase_gtps"
for file in glob.glob(os.path.join(SINGLE_TRACK_DIR, "*.gp*")):
    print(f"processing {file.split('/')[-1]}")
    get_phrases(file, PHRASE_DIR)



In [12]:
PHRASE_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/phrase_gtps"
file_name = "ACDC - Highway To Hell (ver 3)_Guitar 2_6.gp5"
song = guitarpro.parse(os.path.join(PHRASE_DIR, file_name))
poly, mono = poly_vs_mono(song)


BELOW IS SUBJECT TO CHANGE

In [ ]:
JSON_DIR = "/content/drive/MyDrive/UG/proc_phrases_anno"
for file in glob.glob(os.path.join(PHRASE_DIR, "*.gp*")):
    print(f"processing {file.split('/')[-1]}")
    # get_anno(file, JSON_DIR)



In [ ]:
# "Led Zeppelin - Stairway To Heaven_4_34" seems to be empty because of tied notes from the previous phrase


In [29]:
file = "/content/drive/MyDrive/tempochange.gp5"
song = guitarpro.parse(file)
song.tracks[0].measures[1].voices[0].beats[2].effect.mixTableChange.tempo


MixTableItem(value=200, duration=0, allTracks=False)

In [33]:
song.tracks[0].measures[1].voices[0].beats[0].start


4800

In [41]:
song.tracks[0].measures[1].voices[0].beats[1].duration.time


960

In [53]:
song.tracks[0].measures[1].voices[0].beats


In [37]:
song.tracks[0].measures[1].voices[0].beats[3].start


7680

In [58]:
file = "/content/drive/MyDrive/repeat.gp5"
song = guitarpro.parse(file)
song.tracks[0].measures[0].voices[0].beats[-1].duration.time


1920